# Sensitivity Analysis

In [9]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
data_save_dir = "/cluster/home/kamara/syntax-shap/data"

model_name = "gpt2"
threshold_ = 0.5


In [11]:

folder_path = f"/cluster/work/zhang/kamara/syntax-shap/results_ids/scores/{model_name}"  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        if (file.endswith('.pkl')):
            # Extract dataset, model name, and algorithm from the filename
            dataset = parts[-5]
            model_name = parts[-4]
            algorithm = parts[-3]
            seed = parts[-2]
            threshold = parts[-1].split('.pkl')[0]  # Removes the file extension

            if eval(threshold) == threshold_:
                # Construct the full file path
                file_path = os.path.join(folder_path, file)

                # Read the pickle file
                with open(full_path, 'rb') as file:
                    data = pickle.load(file)

                # Assuming the content is a dictionary, create a DataFrame
                df = pd.DataFrame(data)
                # Add the extracted details as new columns
                df['algorithm'] = algorithm
                df['n_token'] = df['input'].str.split().apply(len)
                df['seed'] = seed
                df['dataset'] = dataset

                # Append to the list of dataframes
                df_list.append(df)

# Concatenate all dataframes in the list
df_final_single_threshold = pd.concat(df_list, ignore_index=True)
df_final_single_threshold


,fid_keep_rd,fid_keep,fid_rmv,log_odds_keep,acc_at_k,prob_diff_at_k,input_id,input,algorithm,n_token,seed,dataset
0,0.075490,0.044658,0.087215,-0.717511,0.5,0.372153,195,A rabbit walks with,partition,4,0,negation
1,0.008912,-0.002078,0.008912,0.209548,0.2,0.394719,323,Anna is a mother with a,partition,6,0,negation
2,0.013380,0.013380,0.003083,-9.501555,0.2,0.740757,531,A person has,partition,3,0,negation
3,0.141438,0.152168,0.152168,-11.932746,0.0,0.582714,234,A lion has no,partition,4,0,negation
4,0.035531,0.035531,0.029419,-10.478192,0.1,0.527202,495,A magazine has,partition,3,0,negation
...,...,...,...,...,...,...,...,...,...,...,...,...
64661,0.335945,0.347024,0.800429,-0.568361,0.8,0.091858,2333,Sign language is the most important part of deaf,syntax,9,1,generics
64662,0.005271,0.005271,0.005271,-8.570091,0.0,0.678959,704,Hand washing prevents lots of new,syntax,6,1,generics
64663,0.040082,0.031475,0.040082,-1.538219,0.4,0.649860,5340,Questions are answered quickly even with help ...,syntax,15,1,generics
64664,-0.031858,0.045876,0.045876,-10.733723,0.2,0.367898,73,Water is all around an,syntax,5,1,generics


In [12]:
df = df_final_single_threshold.groupby(['dataset', 'algorithm','seed']).mean().reset_index()

In [13]:
#df = df[df['dataset'] == 'negation']
df.groupby(['dataset', 'algorithm'])['prob_diff_at_k'].agg(['mean', 'std'])

mean       std
dataset    algorithm                    
generics   lime       0.578885  0.007209
           partition  0.550700  0.004737
           random     0.580406       NaN
           shap       0.517579  0.002251
           syntax     0.511797  0.002403
           syntax-w   0.556415  0.007166
negation   lime       0.522205  0.002869
           partition  0.458341  0.014372
           random     0.494329       NaN
           shap       0.485977  0.011432
           syntax     0.463532  0.010265
           syntax-w   0.457848  0.012522
rocstories lime       0.587414  0.001946
           partition  0.541774  0.005594
           random     0.574361       NaN
           shap       0.516675  0.002008
           syntax     0.496512  0.002304
           syntax-w   0.535685  0.002615

In [15]:
df.groupby(['dataset', 'algorithm'])['fid_keep'].agg(['mean', 'std'])

mean       std
dataset    algorithm                    
generics   lime       0.187771  0.004925
           partition  0.176297  0.003825
           random     0.189398       NaN
           shap       0.170076  0.002413
           syntax     0.167111  0.002684
           syntax-w   0.180887  0.002939
negation   lime       0.118911  0.007118
           partition  0.116991  0.005815
           random     0.126044       NaN
           shap       0.117321  0.006821
           syntax     0.120000  0.003141
           syntax-w   0.119340  0.005020
rocstories lime       0.178935  0.002784
           partition  0.161644  0.005283
           random     0.161043       NaN
           shap       0.160010  0.004845
           syntax     0.151618  0.005239
           syntax-w   0.161903  0.006246